In [ ]:
!pip install -q kaggle
!pip install -q mlxtend
!pip install -q sklearn
!pip install -q catboost

!pip install -q optuna

!pip install -q numba==0.53.1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 125.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 63.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files
files.upload()

from google.colab import drive
drive.mount('/content/gdrive')

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c predict-student-performance-from-game-play

! unzip predict-student-performance-from-game-play.zip -d train

Saving kaggle.json to kaggle.json
Mounted at /content/gdrive
100% 482M/484M [00:17<00:00, 32.1MB/s]
100% 484M/484M [00:18<00:00, 28.1MB/s]
Archive:  predict-student-performance-from-game-play.zip
  inflating: train/jo_wilder/__init__.py  
  inflating: train/jo_wilder/competition.cpython-37m-x86_64-linux-gnu.so  
  inflating: train/sample_submission.csv  
  inflating: train/test.csv          
  inflating: train/train.csv         
  inflating: train/train_labels.csv  


In [ ]:
import joblib
import sys
from typing import Dict, List, Tuple
sys.modules['sklearn.externals.joblib'] = joblib

import pandas as pd
from pandas import Series
import pandas.core.algorithms as algos
import numpy as np

from sklearn.feature_selection import mutual_info_classif, f_classif, SelectKBest, chi2, VarianceThreshold
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import f1_score, balanced_accuracy_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

import scipy.stats.stats as stats
import re
import traceback
import string

from tqdm import tqdm

import catboost
from catboost import (
    CatBoostClassifier, 
    EShapCalcType, 
    EFeaturesSelectionAlgorithm,
    Pool,
    EFstrType,
    cv
)

from collections import Counter
import scipy
from numba import njit, jit, float64, float32, int64, int32
import numba

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
dtypes={
    'session_id':'category', 
    'elapsed_time':np.int32,
    'event_name':'category',
    'name':'category',
    'level':np.uint8,
    # 'page':np.int8,
    'room_coor_x':np.float32,
    'room_coor_y':np.float32,
    'screen_coor_x':np.float32,
    'screen_coor_y':np.float32,
    'hover_duration':np.float32,
    'text':'category',
    'fqid':'category',
    'room_fqid':'category',
    'text_fqid':'category',
    'fullscreen':'category',
    'hq':'category',
    'music':'category',
    'level_group':'category'
}

cols_to_use = [
    'session_id', 'index', 'elapsed_time', 'event_name', 'name', 'level',
    'page', 
    'hover_duration', 'fqid', 'room_fqid', 'text_fqid',
    'level_group'
#     TODO: Coordinates, but for me it doesn't tell us anything
#     'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y',
#     Text doesn't tell us anything at all. I will change my mind if smb will proof this
#      'text'
]

In [ ]:
train = pd.read_csv('/content/train/train.csv', dtype=dtypes, usecols=cols_to_use)
test = pd.read_csv('/content/train/test.csv')
submission = pd.read_csv('/content/train/sample_submission.csv')
labels = pd.read_csv('/content/train/train_labels.csv')

In [ ]:
# q1,q2,q3/ lvl0-4
# q4-q13/ lvl5-12
# q14, q15, q16, q17, q18/ 13/22

# Preprocessing

In [ ]:
arr = labels['session_id'].str.split('_').values
ids = []
qs = []

for l in arr:
  ids.append(l[0])
  qs.append(l[1])

labels['id'] = ids
labels['q'] = qs
labels['q'] = labels['q'].str[1:].astype(int)
labels = labels.drop(columns='session_id')

In [ ]:
levels_0_4 = train[(train['level'] >= 0) & (train['level'] <= 4)].drop(columns=['level_group'])
levels_5_12 = train[(train['level'] >= 5) & (train['level'] <= 12)].drop(columns=['level_group'])
levels_13_22 = train[(train['level'] >= 13) & (train['level'] <= 22)].drop(columns=['level_group'])

labels_0_4 = labels[(labels['q'] >= 1) & (labels['q'] <= 3)]
labels_5_12 = labels[(labels['q'] >= 4) & (labels['q'] <= 13)]
labels_13_22 = labels[(labels['q'] >= 14) & (labels['q'] <= 18)]

In [ ]:
levels_0_4 = levels_0_4.sort_values(by=['session_id', 'elapsed_time'])
levels_0_4['previous_action_time'] = levels_0_4.groupby('session_id')['elapsed_time'].shift(1).bfill()
levels_0_4['future_action_time'] = levels_0_4.groupby('session_id')['elapsed_time'].shift(-1).ffill()

levels_0_4['last_action_time_diff'] = levels_0_4['elapsed_time'] - levels_0_4['previous_action_time']
levels_0_4['future_action_time_diff'] = levels_0_4['future_action_time'] - levels_0_4['elapsed_time']

levels_5_12 = levels_5_12.sort_values(by=['session_id', 'elapsed_time'])
levels_5_12['previous_action_time'] = levels_5_12.groupby('session_id')['elapsed_time'].shift(1).bfill()
levels_5_12['future_action_time'] = levels_5_12.groupby('session_id')['elapsed_time'].shift(-1).ffill()

levels_5_12['last_action_time_diff'] = levels_5_12['elapsed_time'] - levels_5_12['previous_action_time']
levels_5_12['future_action_time_diff'] = levels_5_12['future_action_time'] - levels_5_12['elapsed_time']

levels_13_22 = levels_13_22.sort_values(by=['session_id', 'elapsed_time'])
levels_13_22['previous_action_time'] = levels_13_22.groupby('session_id')['elapsed_time'].shift(1).bfill()
levels_13_22['future_action_time'] = levels_13_22.groupby('session_id')['elapsed_time'].shift(-1).ffill()

levels_13_22['last_action_time_diff'] = levels_13_22['elapsed_time'] - levels_13_22['previous_action_time']
levels_13_22['future_action_time_diff'] = levels_13_22['future_action_time'] - levels_13_22['elapsed_time']

In [ ]:
def aggregate_one_level_group(input_df_levels: pd.DataFrame, labels: pd.DataFrame) -> pd.DataFrame:
    aggregated = input_df_levels.groupby('session_id').agg({
            'index' : ['min', 'max', 'count'], 
            'elapsed_time' : ['min', 'max'], 
            'last_action_time_diff': ['max', 'std', 'mean', 'median'], 
            'page': ['count', 'max', 'sum'],
            'hover_duration': ['count', 'max', 'sum']
        })
    aggregated.columns = aggregated.columns.to_flat_index().str.join('_')
    aggregated['hover_duration_max'] = aggregated['hover_duration_max'].fillna(0)
    aggregated['page_max'] = aggregated['page_max'].fillna(-1)

    aggregated_event = input_df_levels.groupby(['session_id', 'event_name']).agg({'index' : ['count'], 
                                                                         'last_action_time_diff' : ['sum', 'max', 'mean', 'median'],
                                                                         'future_action_time_diff' : ['sum', 'max', 'mean', 'median']})
    aggregated_event.columns = aggregated_event.columns.to_flat_index().str.join('_')
    aggregated_event = aggregated_event.unstack()
    aggregated_event.columns = aggregated_event.columns.to_flat_index().str.join('_')

    aggregated_name = input_df_levels.groupby(['session_id', 'name']).agg({'index' : ['count'], 
                                                                  'last_action_time_diff' : ['sum', 'max', 'mean', 'median', 'std'],
                                                                  'future_action_time_diff' : ['sum', 'max', 'mean', 'median']})
    aggregated_name.columns = aggregated_name.columns.to_flat_index().str.join('_')
    aggregated_name = aggregated_name.unstack()
    aggregated_name.columns = aggregated_name.columns.to_flat_index().str.join('_')

    input_df_levels['level'] = input_df_levels['level'].astype(str)
    aggregated_level = input_df_levels.groupby(['session_id', 'level']).agg({'index' : ['count'], 
                                                                        'future_action_time_diff' : ['sum', 'max', 'mean', 'median', 'std']})
    aggregated_level.columns = aggregated_level.columns.to_flat_index().str.join('_')
    aggregated_level = aggregated_level.unstack()
    aggregated_level.columns = aggregated_level.columns.to_flat_index().str.join('_')

    aggregated = pd.merge(aggregated, aggregated_event, left_index=True, right_index=True, how='inner')
    aggregated = pd.merge(aggregated, aggregated_name, left_index=True, right_index=True, how='inner')
    aggregated = pd.merge(aggregated, aggregated_level, left_index=True, right_index=True, how='inner')

    aggregated = pd.merge(aggregated ,labels, right_on='id', left_index=True, how='inner')
    aggregated = aggregated.reset_index(drop=True)

    aggregated = aggregated.fillna(0)
    aggregated['q'] = aggregated['q'].astype('category')
    return aggregated

In [ ]:
first_q_set_aggregated = aggregate_one_level_group(levels_0_4, labels_0_4)
second_q_set_aggregated = aggregate_one_level_group(levels_5_12, labels_5_12)
third_q_set_aggregated = aggregate_one_level_group(levels_13_22, labels_13_22)

In [ ]:
# aggregated_fqid = levels_0_4.groupby(['session_id', 'fqid']).agg({'index' : ['count'], 'future_action_time_diff' : ['sum', 'max', 'mean', 'median', 'std']})
# aggregated_fqid.columns = aggregated_fqid.columns.to_flat_index().str.join('_')
# aggregated_fqid = aggregated_fqid.unstack()
# aggregated_fqid.columns = aggregated_fqid.columns.to_flat_index().str.join('_')

# aggregated_room_fqid = levels_0_4.groupby(['session_id', 'room_fqid']).agg({'index' : ['count'], 'future_action_time_diff' : ['sum', 'max', 'mean', 'median', 'std']})
# aggregated_room_fqid.columns = aggregated_room_fqid.columns.to_flat_index().str.join('_')
# aggregated_room_fqid = aggregated_room_fqid.unstack()
# aggregated_room_fqid.columns = aggregated_room_fqid.columns.to_flat_index().str.join('_')

# aggregated_text_fqid = levels_0_4.groupby(['session_id', 'text_fqid']).agg({'index' : ['count'], 'future_action_time_diff' : ['sum', 'max', 'mean', 'median', 'std']})
# aggregated_text_fqid.columns = aggregated_text_fqid.columns.to_flat_index().str.join('_')
# aggregated_text_fqid = aggregated_text_fqid.unstack()
# aggregated_text_fqid.columns = aggregated_text_fqid.columns.to_flat_index().str.join('_')

# aggregated = pd.merge(aggregated, aggregated_fqid, left_index=True, right_index=True, how='inner')
# aggregated = pd.merge(aggregated, aggregated_room_fqid, left_index=True, right_index=True, how='inner')
# aggregated = pd.merge(aggregated, aggregated_text_fqid, left_index=True, right_index=True, how='inner')

# Model training

## variable search pipeline

In [ ]:
first_q_set_aggregated_sample = first_q_set_aggregated.groupby('correct', group_keys=False).apply(lambda x: x.sample(frac=0.1))
second_q_set_aggregated_sample = second_q_set_aggregated.groupby('correct', group_keys=False).apply(lambda x: x.sample(frac=0.1))
third_q_set_aggregated_sample = third_q_set_aggregated.groupby('correct', group_keys=False).apply(lambda x: x.sample(frac=0.1))

In [ ]:
X_first_sample = first_q_set_aggregated_sample.drop(columns = ['id', 'correct'])
y_first_sample = first_q_set_aggregated_sample['correct']

X_second_sample = second_q_set_aggregated_sample.drop(columns = ['id', 'correct'])
y_second_sample = second_q_set_aggregated_sample['correct']

X_third_sample = third_q_set_aggregated_sample.drop(columns = ['id', 'correct'])
y_third_sample = third_q_set_aggregated_sample['correct']

In [ ]:
X_first = first_q_set_aggregated.drop(columns = ['id', 'correct'])
y_first = first_q_set_aggregated['correct']

X_second = second_q_set_aggregated.drop(columns = ['id', 'correct'])
y_second = second_q_set_aggregated['correct']

X_third = third_q_set_aggregated.drop(columns = ['id', 'correct'])
y_third = third_q_set_aggregated['correct']

In [ ]:
def get_dtype_value_counts(df: pd.DataFrame):
    return df.dtypes.value_counts()

def get_numeric_columns(df: pd.DataFrame):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numerical_cols = list(df.select_dtypes(numerics).columns)
    return numerical_cols

def get_categorical_columns(df: pd.DataFrame):
    catagorical = ['object', 'category']
    catagorical_cols = list(df.select_dtypes(catagorical).columns)
    return catagorical_cols

In [ ]:
def scale_dataframe(df: pd.DataFrame, cols_to_normalize: List[str]) -> pd.DataFrame:
    trans = StandardScaler()
    df[cols_to_normalize] = trans.fit_transform(df[cols_to_normalize])

    scaler = MinMaxScaler()
    df[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])
    return df

def selectionVarianceThreshold(df, variance_threshold=0.0):
    sel = VarianceThreshold(threshold=variance_threshold)
    sel_var=sel.fit_transform(df)
    sel_loc_index = sel.get_support(indices=True)
 
    if len(sel_loc_index)==0:
        return None
    else: 
        return df.iloc[:, sel_loc_index]

def get_best_mutual_information_cols(X, y, top_features=100):
    features_info_limit = 0.0002

    importances = mutual_info_classif(X, y, n_neighbors=3)
    features = pd.Series(importances, X.columns).sort_values(ascending=False)
    features = features[features>features_info_limit]
    if len(features)>=top_features:
        features = features[:top_features]
    return features

def get_chi2_cols(X, y, top_features=100):
    if top_features > len(X.columns):
        top_features='all'

    # Change feats to categorical for chi2 and SelectKBest 
    chi_best_k = SelectKBest(chi2, k=top_features).fit((10*X).astype(int), y)
    scores = chi_best_k.scores_
    chi_best_feats = {feat:scores[index] for (index, feat) in enumerate(list(chi_best_k.get_support()*X.columns)) if feat !=''}
    features = pd.Series(chi_best_feats).sort_values(ascending=False)
    return features

def get_f_classif_cols(X, y, top_features=100):

    if top_features > len(X.columns):
        top_features='all'

    fvalue_selector = SelectKBest(f_classif, k=top_features)
    kbest_anova = fvalue_selector.fit(X, y)
    scores = kbest_anova.scores_

    cols = X.columns
    
    anova_feats = {feat:scores[index] for (index, feat) in enumerate(list(kbest_anova.get_support()*cols)) if feat !=''}
    anova_feats =  pd.Series(anova_feats).sort_values(ascending=False)
    return anova_feats


###############################################################################
max_bin = 20
force_bin = 3
###############################################################################
def data_vars(df1, target):
    # define a binning function
    def mono_bin(Y, X, n = max_bin):
        
        df1 = pd.DataFrame({"X": X, "Y": Y})
        justmiss = df1[['X','Y']][df1.X.isnull()]
        notmiss = df1[['X','Y']][df1.X.notnull()]
        r = 0
        while np.abs(r) < 1:
            try:
                d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
                d2 = d1.groupby('Bucket', as_index=True)
                r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
                n = n - 1 
            except Exception as e:
                n = n - 1

        if len(d2) == 1:
            n = force_bin         
            bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
            if len(np.unique(bins)) == 2:
                bins = np.insert(bins, 0, 1)
                bins[1] = bins[1]-(bins[1]/2)
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
            d2 = d1.groupby('Bucket', as_index=True)
        
        d3 = pd.DataFrame({},index=[])
        d3["MIN_VALUE"] = d2.min().X
        d3["MAX_VALUE"] = d2.max().X
        d3["COUNT"] = d2.count().Y
        d3["EVENT"] = d2.sum().Y
        d3["NONEVENT"] = d2.count().Y - d2.sum().Y
        d3=d3.reset_index(drop=True)
        
        if len(justmiss.index) > 0:
            d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
            d4["MAX_VALUE"] = np.nan
            d4["COUNT"] = justmiss.count().Y
            d4["EVENT"] = justmiss.sum().Y
            d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
            d3 = d3.append(d4,ignore_index=True)
        
        d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
        d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
        d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
        d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
        d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
        d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
        d3["VAR_NAME"] = "VAR"
        d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
        d3 = d3.replace([np.inf, -np.inf], 0)
        d3.IV = d3.IV.sum()
        
        return(d3)

    def char_bin(Y, X):
        
        df1 = pd.DataFrame({"X": X, "Y": Y})
        justmiss = df1[['X','Y']][df1.X.isnull()]
        notmiss = df1[['X','Y']][df1.X.notnull()]    
        df2 = notmiss.groupby('X',as_index=True)
        
        d3 = pd.DataFrame({},index=[])
        d3["COUNT"] = df2.count().Y
        d3["MIN_VALUE"] = df2.sum().Y.index
        d3["MAX_VALUE"] = d3["MIN_VALUE"]
        d3["EVENT"] = df2.sum().Y
        d3["NONEVENT"] = df2.count().Y - df2.sum().Y
        
        if len(justmiss.index) > 0:
            d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
            d4["MAX_VALUE"] = np.nan
            d4["COUNT"] = justmiss.count().Y
            d4["EVENT"] = justmiss.sum().Y
            d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
            d3 = d3.append(d4,ignore_index=True)
        
        d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
        d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
        d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
        d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
        d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
        d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
        d3["VAR_NAME"] = "VAR"
        d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
        d3 = d3.replace([np.inf, -np.inf], 0)
        d3.IV = d3.IV.sum()
        d3 = d3.reset_index(drop=True)
        
        return(d3)
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = iv_df.groupby('VAR_NAME').IV.max()
    return iv


def get_top_features_with_scores(X: pd.DataFrame, y: pd.Series, cols_to_normalize=None, top_features=50) -> Dict[str, List[Tuple]]:
    variance_threshold = 0.0001

    choosed_features = {}
    cols_to_normalize = cols_to_normalize if cols_to_normalize is not None else get_numeric_columns(X)

    X = scale_dataframe(X, cols_to_normalize)
    print('No cols after scaling:', len(X.columns))
    X = selectionVarianceThreshold(X, variance_threshold) if variance_threshold else X
    print('No cols after variance threshold:', len(X.columns))

    numeric_cols = get_numeric_columns(X)
    print('Final No numeric cols:', len(numeric_cols))

    choosed_features['mutual_info'] = get_best_mutual_information_cols(X, y, top_features=top_features)
    print('Mutual INFO calculated')

    print(len(X.columns))
    choosed_features['chi2_score'] = get_chi2_cols(X[numeric_cols], y, top_features=top_features)
    print('CHI2 Score calculated')

    choosed_features['f_classif_score'] = get_f_classif_cols(X, y, top_features=top_features)
    print('F_classif Score calculated')

    choosed_features['IV_score'] = get_f_classif_cols(X[numeric_cols], y, top_features=top_features)
    print('IV Score calculated')

    return choosed_features

In [ ]:
choosed_features = get_top_features_with_scores(X_third, y_third, top_features=35)

No cols after scaling: 235
No cols after variance threshold: 211
Final No numeric cols: 210
Mutual INFO calculated
211
CHI2 Score calculated
F_classif Score calculated
IV Score calculated


In [ ]:
def get_sum_of_choosed_features(choosed_features: Dict[str, pd.Series]) -> List[str]:
    features = list()
    for key, series in choosed_features.items():
        features.extend(list(series.keys()))

    features = list(set(features))
    return features


In [ ]:
all_features = get_sum_of_choosed_features(choosed_features)
len(all_features)

73

# Inference

In [ ]:
def train_catboost_on_cv(X, y, threshold=None, log=None, cat_features=None, calculate_feature_importances = True, params=None):
    
    default_params = {
    'iterations': 100,
    'depth': 4,
    'loss_function': 'Logloss',
    'auto_class_weights': 'Balanced',
    'task_type': "GPU",
    'verbose': False,
    }

    if params is None:
        params = default_params

    k_fold = StratifiedKFold(n_splits=5, shuffle=True)
    folds = k_fold.split(X, y)

    train_scores = []
    test_scores = []
    
    recalls = []
    precisions = []

    f1_train_scores = []
    f1_test_scores = []

    feature_importances = []


    for k, (train, test) in enumerate(folds, start=1):
        train_pool = Pool(X.iloc[train, :], y.iloc[train], cat_features=cat_features)
        test_pool = Pool(X.iloc[test, :], y.iloc[test], cat_features=cat_features)

        model = CatBoostClassifier(**params)
        model.fit(train_pool, eval_set=test_pool, use_best_model=True, verbose=params['verbose']) 

        train_preds = model.predict(X.iloc[train, :])
        train_score = balanced_accuracy_score(y.iloc[train].values, train_preds)
        train_scores.append(train_score)

        if threshold:
            y_preds = (model.predict_proba(X.iloc[test, :])[:, 1] >= threshold).astype(bool)
        else:
            y_preds = model.predict(X.iloc[test, :])
            
        cm = confusion_matrix(y.iloc[test].values, y_preds)

        test_score = balanced_accuracy_score(y.iloc[test].values, y_preds)
        test_scores.append(test_score)

        recall = cm[1][1]/(cm[1][1]+cm[1][0])
        recalls.append(recall)
        precision = cm[1][1]/(cm[0][1]+cm[1][1])
        precisions.append(precision)

        f1_score_train = f1_score(y.iloc[train].values, train_preds, average='macro')
        f1_score_test = f1_score(y.iloc[test].values, y_preds, average='macro')

        f1_train_scores.append(f1_score_train)
        f1_test_scores.append(f1_score_test)

        if calculate_feature_importances:
            print('Calculating feature importance')
            importances = model.get_feature_importance(data=train_pool, reference_data=test_pool, 
                                        type=EFstrType.FeatureImportance,
                                        prettified=True, verbose=True)
            feature_importances.append(importances)

        if log:
            print(f'Fold: {k}')
            print(f'best_iteration: {model.best_iteration_:.3f}')
            print(f'best_score: {model.best_score_}')
            print(f'train_balanced_accuracy: {train_score:.3f},  test_balanced_accuracy: {test_score:.3f}')
            print(f'train_f1_score: {f1_score_train:.3f},  test_f1_score: {f1_score_test:.3f}')
            print(f'precision: {precision:.3f}, recall: {recall:.3f}')
            # print(cm)
            print()

    return_scores = {}

    return_scores['mean_train_score'] = np.mean(train_scores)
    return_scores['mean_test_score'] = np.mean(test_scores)
    return_scores['mean_recall'] = np.mean(recalls)
    return_scores['mean_precision'] = np.mean(precisions)
    return_scores['mean_f1_train_scores'] = np.mean(f1_score_train)
    return_scores['mean_f1_test_scores'] = np.mean(f1_score_test)

    return return_scores, feature_importances





In [ ]:
def f1_avg(preds, labels): # my calc of macro F1 - way faster
    c11 = (preds + labels == 2).mean()
    c00 = (preds + labels == 0).mean()
    e = 1 - (1 - c00 - c11) / (1 - (c00 - c11)**2)
    return e

class F1Metric(object):
    global threshold
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight) -> Tuple[float, float]:
        approx = approxes[0]
        approx = [1 / (1 + np.exp(-x)) for x in approx]
        pred_labels = [np.float32(1.0) if x > threshold else np.float32(0.0) for x in approx]
        f1 = f1_avg(pred_labels,target)
        return (f1, 1.0)

In [ ]:
# X = third_q_set_aggregated.drop(columns = ['id', 'correct'])
# X = X[all_features]
# y = third_q_set_aggregated['correct']

X, y = X_first, y_first

In [ ]:
threshold=0.5

params = {
    'iterations': 1000,
    'depth': 4,
    'od_type': 'IncToDec',
    'od_pval': 0.97,
    # 'grow_policy': "Lossguide",
    'loss_function': 'Logloss',
    'l2_leaf_reg': 1,
    'auto_class_weights': 'Balanced',
    # 'eval_metric': F1Metric(),
    'task_type': "GPU",
    'verbose': True,
}
categorical_columns = get_categorical_columns(X)
train_catboost_on_cv(X, y, log=True, cat_features=categorical_columns, calculate_feature_importances=False, threshold=0.5, params=params)

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
50:	learn: 0.5095013	test: 0.5000686	best: 0.5000686 (50)	total: 1.74s	remaining: 32.4s
51:	learn: 0.5086711	test: 0.4992159	best: 0.4992159 (51)	total: 1.77s	remaining: 32.2s
52:	learn: 0.5081891	test: 0.4987965	best: 0.4987965 (52)	total: 1.79s	remaining: 32s
53:	learn: 0.5074285	test: 0.4979672	best: 0.4979672 (53)	total: 1.82s	remaining: 31.9s
54:	learn: 0.5067926	test: 0.4973464	best: 0.4973464 (54)	total: 1.85s	remaining: 31.7s
55:	learn: 0.5061207	test: 0.4967222	best: 0.4967222 (55)	total: 1.87s	remaining: 31.6s
56:	learn: 0.5055645	test: 0.4961174	best: 0.4961174 (56)	total: 1.9s	remaining: 31.4s
57:	learn: 0.5048346	test: 0.4952304	best: 0.4952304 (57)	total: 1.93s	remaining: 31.3s
58:	learn: 0.5041054	test: 0.4944021	best: 0.4944021 (58)	total: 1.95s	remaining: 31.2s
59:	learn: 0.5033877	test: 0.4936199	best: 0.4936199 (59)	total: 1.98s	remaining: 31s
60:	learn: 0.5031708	test: 0.4934017	best: 0.4934017 (60)	to

({'mean_train_score': 0.7989278300945115,
  'mean_test_score': 0.7782403902406456,
  'mean_recall': 0.762992408388268,
  'mean_precision': 0.9638779560427594,
  'mean_f1_train_scores': 0.6642172419774068,
  'mean_f1_test_scores': 0.6475322025177311},
 [])

In [ ]:
params = {
    'iterations': 350,
    'depth': 10,
    'grow_policy': "Lossguide",
    'loss_function': 'Logloss',
    'l2_leaf_reg': 5,
    'auto_class_weights': 'Balanced',
    # 'task_type': "GPU",
    'verbose': False,
}

categorical_columns = get_categorical_columns(X)
all_scores_f1 = []
for i in tqdm(range(5), position=0, mininterval=1.0):
    scores, feature_importances = train_catboost_on_cv(X, y, log=True, cat_features=categorical_columns, calculate_feature_importances=False, threshold=0.5, params=params)
    all_scores_f1.append(scores['mean_f1_test_scores'])
print()
print(all_scores_f1)
print(np.mean(all_scores_f1)) 

  0%|          | 0/5 [00:00<?, ?it/s]

Fold: 1
best_iteration: 119.000
best_score: {'learn': {'Logloss': 0.10288017702020268}, 'validation': {'Logloss': 0.4996098274817867}}
train_balanced_accuracy: 0.880,  test_balanced_accuracy: 0.763
train_f1_score: 0.757,  test_f1_score: 0.680
precision: 0.951, recall: 0.822



  0%|          | 0/5 [01:15<?, ?it/s]


KeyboardInterrupt: ignored

## Feature elimination

In [ ]:
threshold=0.5

def select_features_with_kfold(X_train: pd.DataFrame, y_train: pd.Series, num_features_to_select: int, steps: int, cat_features) -> Tuple[float, float]:

  k_fold = StratifiedKFold(n_splits=5, shuffle=True)
  folds = k_fold.split(X_train, y_train)
      
  algorithm = EFeaturesSelectionAlgorithm.RecursiveByShapValues
  shap_calc_type = EShapCalcType.Regular
  summaries = []

  for k, (train, test) in enumerate(folds, start=1):
    train_pool = Pool(X_train.iloc[train, :], y_train.iloc[train], cat_features=cat_features)
    test_pool = Pool(X_train.iloc[test, :], y_train.iloc[test], cat_features=cat_features)

    params = {
        'iterations': 0,
        'depth': 8,
        'grow_policy': "Lossguide",
        'loss_function': 'Logloss',
        'l2_leaf_reg': 5,
        'auto_class_weights': 'Balanced',
        'eval_metric': F1Metric(),
        'verbose': False,
    }

    model = CatBoostClassifier(**params)

    summary = model.select_features(
      train_pool,
      eval_set=test_pool,
      features_for_select=list(range(X_train.shape[1])),
      num_features_to_select = num_features_to_select,
      steps=steps,
      algorithm=algorithm,
      shap_calc_type=shap_calc_type, 
      train_final_model=False,
    )
    summaries.append(summary)

  return summaries

In [ ]:
categorical_columns = get_categorical_columns(X_third[all_features])
summaries = select_features_with_kfold(X_third[all_features], y_third, num_features_to_select=30, steps=4, cat_features=categorical_columns)

Step #1 out of 4

bestTest = 0.6272798862
bestIteration = 345

Shrink model to first 346 iterations.
Feature #51 eliminated
Feature #30 eliminated
Feature #70 eliminated
Feature #44 eliminated
Feature #58 eliminated
Feature #49 eliminated
Feature #67 eliminated
Feature #23 eliminated
Feature #63 eliminated
Feature #50 eliminated
Feature #56 eliminated
Feature #11 eliminated
Feature #16 eliminated
Feature #46 eliminated
Feature #3 eliminated
Step #2 out of 4

bestTest = 0.6264088284
bestIteration = 349

Feature #7 eliminated
Feature #17 eliminated
Feature #66 eliminated
Feature #71 eliminated
Feature #24 eliminated
Feature #8 eliminated
Feature #19 eliminated
Feature #1 eliminated
Feature #13 eliminated
Feature #10 eliminated
Feature #38 eliminated
Step #3 out of 4

bestTest = 0.6273249519
bestIteration = 348

Shrink model to first 349 iterations.
Feature #27 eliminated
Feature #64 eliminated
Feature #47 eliminated
Feature #14 eliminated
Feature #40 eliminated
Feature #22 eliminated
Fea

In [ ]:
selected_features = [summary['selected_features_names'] for summary in summaries]
indiv_selected_features = list(set(sum(selected_features, [])))
indiv_selected_features

['index_count_close',
 'future_action_time_diff_mean_21',
 'last_action_time_diff_sum_person_click',
 'index_count_22',
 'last_action_time_diff_mean_person_click',
 'last_action_time_diff_mean_object_hover',
 'index_max',
 'index_count_13',
 'future_action_time_diff_std_21',
 'last_action_time_diff_median_undefined',
 'index_count_open',
 'future_action_time_diff_median_undefined',
 'last_action_time_diff_std_undefined',
 'index_count_undefined',
 'last_action_time_diff_median',
 'index_count_18',
 'future_action_time_diff_std_18',
 'future_action_time_diff_mean_navigate_click',
 'last_action_time_diff_median_close',
 'future_action_time_diff_sum_close',
 'last_action_time_diff_max',
 'last_action_time_diff_median_navigate_click',
 'last_action_time_diff_sum_cutscene_click',
 'index_count_15',
 'hover_duration_count',
 'future_action_time_diff_mean_13',
 'future_action_time_diff_mean_person_click',
 'index_count_person_click',
 'page_sum',
 'index_count_16',
 'elapsed_time_max',
 'inde

In [ ]:
categorical_columns = get_categorical_columns(X_second[indiv_selected_features])
summaries = select_features_with_kfold(X_second[indiv_selected_features], y_second, num_features_to_select=20, steps=4, cat_features=categorical_columns)

Step #1 out of 4

bestTest = 0.6769557826
bestIteration = 148

Shrink model to first 149 iterations.
Feature #28 eliminated
Feature #26 eliminated
Feature #23 eliminated
Feature #25 eliminated
Step #2 out of 4

bestTest = 0.6763824439
bestIteration = 146

Shrink model to first 147 iterations.
Feature #4 eliminated
Feature #30 eliminated
Feature #8 eliminated
Feature #3 eliminated
Step #3 out of 4

bestTest = 0.6751162589
bestIteration = 143

Shrink model to first 144 iterations.
Feature #2 eliminated
Feature #33 eliminated
Feature #14 eliminated
Step #4 out of 4

bestTest = 0.6748148028
bestIteration = 148

Shrink model to first 149 iterations.
Feature #9 eliminated
Feature #12 eliminated
Feature #21 eliminated
Step #1 out of 4

bestTest = 0.6774032138
bestIteration = 149

Feature #4 eliminated
Feature #16 eliminated
Feature #7 eliminated
Feature #25 eliminated
Step #2 out of 4

bestTest = 0.677389682
bestIteration = 147

Shrink model to first 148 iterations.
Feature #33 eliminated
Fea

CatBoostError: ignored

In [ ]:
selected_features = [summary['selected_features_names'] for summary in summaries]
indiv_selected_features_2 = list(set(sum(selected_features, [])))

indiv_selected_features_2 = ['future_action_time_diff_sum_12',
 'last_action_time_diff_median_person_click',
 'index_count_12',
 'future_action_time_diff_sum_close',
 'future_action_time_diff_sum_object_click',
 'future_action_time_diff_median_basic',
 'index_count_11',
 'last_action_time_diff_mean_observation_click',
 'future_action_time_diff_median_10',
 'last_action_time_diff_sum_map_click',
 'index_count_undefined',
 'q',
 'future_action_time_diff_std_10',
 'index_count_object_click',
 'index_count_object_hover',
 'last_action_time_diff_std_close',
 'hover_duration_count',
 'index_count_9',
 'index_count_basic',
 'last_action_time_diff_median_cutscene_click',
 'last_action_time_diff_max_observation_click',
 'future_action_time_diff_max_observation_click',
 'index_count',
 'index_max',
 'last_action_time_diff_sum_object_click',
 'last_action_time_diff_median_close',
 'future_action_time_diff_sum_undefined',
 'index_count_map_click',
 'last_action_time_diff_median_observation_click',
 'future_action_time_diff_sum_navigate_click',
 'last_action_time_diff_sum_cutscene_click',
 'future_action_time_diff_median_person_click',
 'future_action_time_diff_max_8',
 'future_action_time_diff_sum_11']

['future_action_time_diff_max_navigate_click',
 'future_action_time_diff_sum_open',
 'future_action_time_diff_median_1',
 'future_action_time_diff_max_4',
 'future_action_time_diff_sum_cutscene_click',
 'index_count_3',
 'future_action_time_diff_max_undefined',
 'future_action_time_diff_median_basic',
 'last_action_time_diff_max_basic',
 'future_action_time_diff_sum_navigate_click',
 'index_max',
 'last_action_time_diff_median_checkpoint',
 'index_count',
 'future_action_time_diff_median_cutscene_click',
 'future_action_time_diff_median_2',
 'future_action_time_diff_sum_undefined',
 'last_action_time_diff_max_checkpoint',
 'future_action_time_diff_sum_3',
 'last_action_time_diff_median_basic',
 'last_action_time_diff_mean_checkpoint',
 'page_count',
 'last_action_time_diff_sum_checkpoint',
 'index_count_object_click',
 'last_action_time_diff_median_cutscene_click',
 'last_action_time_diff_median_person_click',
 'q',
 'future_action_time_diff_sum_4',
 'index_count_basic',
 'last_action_

In [ ]:
# indiv_selected_features_2_first_model = ['future_action_time_diff_max_navigate_click',
#  'future_action_time_diff_sum_open',
#  'future_action_time_diff_median_1',
#  'future_action_time_diff_max_4',
#  'future_action_time_diff_sum_cutscene_click',
#  'index_count_3',
#  'future_action_time_diff_max_undefined',
#  'future_action_time_diff_median_basic',
#  'last_action_time_diff_max_basic',
#  'future_action_time_diff_sum_navigate_click',
#  'index_max',
#  'last_action_time_diff_median_checkpoint',
#  'index_count',
#  'future_action_time_diff_median_cutscene_click',
#  'future_action_time_diff_median_2',
#  'future_action_time_diff_sum_undefined',
#  'last_action_time_diff_max_checkpoint',
#  'future_action_time_diff_sum_3',
#  'last_action_time_diff_median_basic',
#  'last_action_time_diff_mean_checkpoint',
#  'page_count',
#  'last_action_time_diff_sum_checkpoint',
#  'index_count_object_click',
#  'last_action_time_diff_median_cutscene_click',
#  'last_action_time_diff_median_person_click',
#  'q',
#  'future_action_time_diff_sum_4',
#  'index_count_basic',
#  'last_action_time_diff_mean_cutscene_click']

# indiv_selected_features_2_second_model = ['future_action_time_diff_sum_12',
# 'last_action_time_diff_median_person_click',
# 'index_count_12',
# 'future_action_time_diff_sum_close',
# 'future_action_time_diff_sum_object_click',
# 'future_action_time_diff_median_basic',
# 'index_count_11',
# 'last_action_time_diff_mean_observation_click',
# 'future_action_time_diff_median_10',
# 'last_action_time_diff_sum_map_click',
# 'index_count_undefined',
# 'q',
# 'future_action_time_diff_std_10',
# 'index_count_object_click',
# 'index_count_object_hover',
# 'last_action_time_diff_std_close',
# 'hover_duration_count',
# 'index_count_9',
# 'index_count_basic',
# 'last_action_time_diff_median_cutscene_click',
# 'last_action_time_diff_max_observation_click',
# 'future_action_time_diff_max_observation_click',
# 'index_count',
# 'index_max',
# 'last_action_time_diff_sum_object_click',
# 'last_action_time_diff_median_close',
# 'future_action_time_diff_sum_undefined',
# 'index_count_map_click',
# 'last_action_time_diff_median_observation_click',
# 'future_action_time_diff_sum_navigate_click',
# 'last_action_time_diff_sum_cutscene_click',
# 'future_action_time_diff_median_person_click',
# 'future_action_time_diff_max_8',
# 'future_action_time_diff_sum_11']


# indiv_selected_features_2_third_model = ['index_count_close',
#  'future_action_time_diff_mean_21',
#  'last_action_time_diff_sum_person_click',
#  'index_count_22',
#  'last_action_time_diff_mean_person_click',
#  'last_action_time_diff_mean_object_hover',
#  'index_max',
#  'index_count_13',
#  'future_action_time_diff_std_21',
#  'last_action_time_diff_median_undefined',
#  'index_count_open',
#  'future_action_time_diff_median_undefined',
#  'last_action_time_diff_std_undefined',
#  'index_count_undefined',
#  'last_action_time_diff_median',
#  'index_count_18',
#  'future_action_time_diff_std_18',
#  'future_action_time_diff_mean_navigate_click',
#  'last_action_time_diff_median_close',
#  'future_action_time_diff_sum_close',
#  'last_action_time_diff_max',
#  'last_action_time_diff_median_navigate_click',
#  'last_action_time_diff_sum_cutscene_click',
#  'index_count_15',
#  'hover_duration_count',
#  'future_action_time_diff_mean_13',
#  'future_action_time_diff_mean_person_click',
#  'index_count_person_click',
#  'page_sum',
#  'index_count_16',
#  'elapsed_time_max',
#  'index_min',
#  'index_count_map_click',
#  'page_count',
#  'index_count_17',
#  'index_count_21',
#  'index_count_navigate_click',
#  'future_action_time_diff_sum_basic',
#  'index_count_map_hover',
#  'future_action_time_diff_sum_21',
#  'future_action_time_diff_std_13',
#  'index_count_object_click',
#  'index_count_19',
#  'index_count',
#  'last_action_time_diff_median_observation_click',
#  'elapsed_time_min',
#  'last_action_time_diff_sum_basic',
#  'q',
#  'index_count_basic',
#  'index_count_20',
#  'last_action_time_diff_sum_close',
#  'index_count_object_hover',
#  'future_action_time_diff_sum_undefined',
#  'future_action_time_diff_median_navigate_click',
#  'future_action_time_diff_sum_19',
#  'future_action_time_diff_mean_undefined']

## Tuning

In [ ]:
import numpy as np
import optuna
from optuna.integration import CatBoostPruningCallback

import catboost as cb
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# X = X_third[indiv_selected_features_2_third_model]
X = X_third
y = y_third

def objective(trial: optuna.Trial, cat_features=None) -> float:
    global threshold
    cat_features = get_categorical_columns(X)

    k_fold = StratifiedKFold(n_splits=4, shuffle=True)
    folds = k_fold.split(X, y)

    f1_scores = []

    threshold = trial.suggest_float('threshold', 0.1, 0.6)

    param = {
        "objective": "Logloss",
        'iterations': trial.suggest_int("iterations", 50, 450),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.1, 1.0, log=True),
        "depth": trial.suggest_int("depth", 5, 11),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 3,20),
        "boosting_type": "Plain",
        "bootstrap_type": "MVS",
        'random_strength': trial.suggest_int("random_strength", 1, 5),
        'grow_policy': trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", 'Lossguide']),
        # 'grow_policy': "SymmetricTree",
        'auto_class_weights': 'Balanced',
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 1000, step=100),
        'task_type': "CPU",
        "eval_metric": F1Metric(),
        'verbose': False,
    }

    for k, (train, test) in enumerate(folds, start=1):
        train_pool = Pool(X.iloc[train, :], y.iloc[train], cat_features=cat_features)
        test_pool = Pool(X.iloc[test, :], y.iloc[test], cat_features=cat_features)

        model = CatBoostClassifier(**param)
        # pruning_callback = CatBoostPruningCallback(trial, "LogLoss")
        model.fit(
            train_pool,
            eval_set=test_pool,
            verbose=0,
            early_stopping_rounds=100,
            use_best_model=True
            # callbacks=[pruning_callback],
        )

        # pruning_callback.check_pruned()

        y_preds = (model.predict_proba(X.iloc[test, :])[:, 1] >= threshold).astype(bool)
        f1_score_test = f1_score(y.iloc[test].values, y_preds, average='macro')
        print(f1_score_test)

        f1_scores.append(f1_score_test)

    return np.mean(f1_scores)

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=None, n_jobs=1, show_progress_bar=True)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-02-26 14:26:21,550] A new study created in memory with name: no-name-9bf1ca22-5804-441d-8084-4b4feb53a5a8
/usr/local/lib/python3.8/dist-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

0.5849469939562666
0.5894854297608236
0.5845131129879629
0.5948237313970652
[I 2023-02-26 14:36:51,004] Trial 0 finished with value: 0.5884423170255295 and parameters: {'threshold': 0.5659055397344662, 'iterations': 444, 'colsample_bylevel': 0.2050778248247302, 'depth': 8, 'l2_leaf_reg': 8, 'random_strength': 1, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 410}. Best is trial 0 with value: 0.5884423170255295.
0.6181305391075506
0.6151142920905261
0.6176246927423878
0.625549613618322
[I 2023-02-26 14:39:32,786] Trial 1 finished with value: 0.6191047843896966 and parameters: {'threshold': 0.35716723851634236, 'iterations': 91, 'colsample_bylevel': 0.3698070239528951, 'depth': 9, 'l2_leaf_reg': 14, 'random_strength': 3, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 910}. Best is trial 1 with value: 0.6191047843896966.
0.5709700232294175
0.5698850654138322
0.5766902476989948
0.5751179231208758
[I 2023-02-26 14:43:32,415] Trial 2 finished with value: 0.5731658148657801 and paramete

In [ ]:
# Trial 106 finished with value: 0.7064968084080109 and parameters: 
# First set
# {
#     'threshold': 0.28256667586733086, 
#     'iterations': 241, 
#     'colsample_bylevel': 0.53262721224417, 
#     'depth': 9, 
#     'l2_leaf_reg': 9, 
#     'boosting_type': 'Plain', 
#     'bootstrap_type': 'MVS', 
#     'random_strength': 1, 
#     'min_data_in_leaf': 310
#  }
# Second Set
{
    'threshold': 0.43831901478524554, 
    'iterations': 411, 
    'colsample_bylevel': 0.6875495483997605, 
    'depth': 11, 
    'l2_leaf_reg': 10, 
    'random_strength': 1, 
    'min_data_in_leaf': 110
 }
# Third Set
{
    'threshold': 0.43765187371446845, 
    'iterations': 313, 
    'colsample_bylevel': 0.3641542533411468, 
    'depth': 11, 'l2_leaf_reg': 15, 
    'random_strength': 5, 
    'min_data_in_leaf': 410
}
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 121
Best trial:
  Value: 0.7064968084080109
  Params: 
    threshold: 0.28256667586733086
    iterations: 241
    colsample_bylevel: 0.53262721224417
    depth: 9
    l2_leaf_reg: 9
    boosting_type: Plain
    bootstrap_type: MVS
    random_strength: 1
    min_data_in_leaf: 310


## Final Models

In [ ]:
indiv_selected_features_2 = ['future_action_time_diff_max_navigate_click',
 'future_action_time_diff_sum_open',
 'future_action_time_diff_sum_cutscene_click',
 'future_action_time_diff_max_undefined',
 'future_action_time_diff_median_basic',
 'last_action_time_diff_max_basic',
 'future_action_time_diff_sum_navigate_click',
 'index_max',
 'last_action_time_diff_median_checkpoint',
 'index_count',
 'future_action_time_diff_median_cutscene_click',
 'future_action_time_diff_sum_undefined',
 'last_action_time_diff_max_checkpoint',
 'last_action_time_diff_median_basic',
 'last_action_time_diff_mean_checkpoint',
 'page_count',
 'last_action_time_diff_sum_checkpoint',
 'index_count_object_click',
 'last_action_time_diff_median_cutscene_click',
 'last_action_time_diff_median_person_click',
 'q',
 'index_count_basic',
 'last_action_time_diff_mean_cutscene_click']


In [ ]:
threshold = 0.28256667586733086
params = {
    "objective": "Logloss",
    'iterations': 241, 
    'colsample_bylevel': 0.53262721224417, 
    'depth': 9, 
    'l2_leaf_reg': 9,
    'boosting_type': 'Plain', 
    'bootstrap_type': 'MVS', 
    'random_strength': 1, 
    'min_data_in_leaf': 310,
    'grow_policy': "SymmetricTree",
    'auto_class_weights': 'Balanced',
    'task_type': "CPU",
    "eval_metric": F1Metric(),
 }

categorical_columns = get_categorical_columns(X_first)

scores, feature_importances = train_catboost_on_cv(X_first[indiv_selected_features_2], y_first, log=True, cat_features=categorical_columns, calculate_feature_importances=False, threshold=threshold, params=params)

CatBoostError: ignored

In [ ]:
categorical_columns = get_categorical_columns(X_second)
scores, feature_importances = train_catboost_on_cv(X_second[indiv_selected_features_2], y_second, log=True, cat_features=categorical_columns, calculate_feature_importances=False, threshold=threshold, params=params)

Fold: 1
best_iteration: 240.000
best_score: {'learn': {'Logloss': 0.5657086213300974, 'F1Metric': 0.6358382358624717}, 'validation': {'Logloss': 0.5848562082195499, 'F1Metric': 0.6201521433581385}}
train_balanced_accuracy: 0.703,  test_balanced_accuracy: 0.622
train_f1_score: 0.692,  test_f1_score: 0.620
precision: 0.712, recall: 0.939

Fold: 2
best_iteration: 226.000
best_score: {'learn': {'Logloss': 0.5654463940361972, 'F1Metric': 0.6398287713069978}, 'validation': {'Logloss': 0.5840105195797709, 'F1Metric': 0.6197779167250053}}
train_balanced_accuracy: 0.702,  test_balanced_accuracy: 0.621
train_f1_score: 0.692,  test_f1_score: 0.620
precision: 0.712, recall: 0.937

Fold: 3
best_iteration: 240.000
best_score: {'learn': {'Logloss': 0.5680576469353525, 'F1Metric': 0.635471064788785}, 'validation': {'Logloss': 0.5778353927733326, 'F1Metric': 0.6282269271775692}}
train_balanced_accuracy: 0.701,  test_balanced_accuracy: 0.628
train_f1_score: 0.689,  test_f1_score: 0.628
precision: 0.716,

KeyboardInterrupt: ignored

In [ ]:
categorical_columns = get_categorical_columns(X_third)
scores, feature_importances = train_catboost_on_cv(X_third[indiv_selected_features_2], y_third, log=True, cat_features=categorical_columns, calculate_feature_importances=False, threshold=threshold, params=params)

Fold: 1
best_iteration: 220.000
best_score: {'learn': {'Logloss': 0.558000014325965, 'F1Metric': 0.6044348374551387}, 'validation': {'Logloss': 0.5927763169642913, 'F1Metric': 0.5811869289210114}}
train_balanced_accuracy: 0.692,  test_balanced_accuracy: 0.584
train_f1_score: 0.657,  test_f1_score: 0.581
precision: 0.751, recall: 0.952

Fold: 2
best_iteration: 226.000
best_score: {'learn': {'Logloss': 0.5599058441188897, 'F1Metric': 0.604979118194229}, 'validation': {'Logloss': 0.5865610391385647, 'F1Metric': 0.5862504108959918}}
train_balanced_accuracy: 0.694,  test_balanced_accuracy: 0.587
train_f1_score: 0.659,  test_f1_score: 0.586
precision: 0.753, recall: 0.954



CatBoostError: ignored

In [ ]:
def train_final_model(X, y, cat_features=None, params=None):
    train_pool = Pool(X, y, cat_features=cat_features)

    model = CatBoostClassifier(**params)
    model.fit(train_pool, use_best_model=False, verbose=True) 
    return model

In [ ]:
params = {
    "objective": "Logloss",
    'iterations': 235, 
    'colsample_bylevel': 0.53262721224417, 
    'depth': 9, 
    'l2_leaf_reg': 9,
    'boosting_type': 'Plain', 
    'bootstrap_type': 'MVS', 
    'random_strength': 1, 
    'min_data_in_leaf': 310,
    'grow_policy': "SymmetricTree",
    'auto_class_weights': 'Balanced',
    'task_type': "CPU",
    "eval_metric": F1Metric(),
 }

model = train_final_model(X_first, y_first, cat_features=['q'], params=params)
model.save_model('first_questions_model.cbm')

0:	learn: 0.4676078	total: 739ms	remaining: 2m 52s
1:	learn: 0.4676078	total: 1.29s	remaining: 2m 30s
2:	learn: 0.4676078	total: 1.79s	remaining: 2m 18s
3:	learn: 0.4676078	total: 2.3s	remaining: 2m 12s
4:	learn: 0.4676078	total: 2.77s	remaining: 2m 7s
5:	learn: 0.4676078	total: 3.25s	remaining: 2m 4s
6:	learn: 0.4676078	total: 3.92s	remaining: 2m 7s
7:	learn: 0.4676078	total: 4.88s	remaining: 2m 18s
8:	learn: 0.4676078	total: 5.74s	remaining: 2m 24s
9:	learn: 0.4676078	total: 6.62s	remaining: 2m 28s
10:	learn: 0.4676078	total: 7.45s	remaining: 2m 31s
11:	learn: 0.4676078	total: 8.37s	remaining: 2m 35s
12:	learn: 0.4676078	total: 9.19s	remaining: 2m 36s
13:	learn: 0.4685653	total: 9.66s	remaining: 2m 32s
14:	learn: 0.4711897	total: 10.1s	remaining: 2m 28s
15:	learn: 0.4770861	total: 10.6s	remaining: 2m 25s
16:	learn: 0.4888594	total: 11.1s	remaining: 2m 22s
17:	learn: 0.5001984	total: 11.7s	remaining: 2m 20s
18:	learn: 0.5120045	total: 12.1s	remaining: 2m 17s
19:	learn: 0.5255140	total

KeyboardInterrupt: ignored

In [ ]:
params = {
    "objective": "Logloss",
    'iterations': 230, 
    'colsample_bylevel': 0.53262721224417, 
    'depth': 9, 
    'l2_leaf_reg': 9,
    'boosting_type': 'Plain', 
    'bootstrap_type': 'MVS', 
    'random_strength': 1, 
    'min_data_in_leaf': 310,
    'grow_policy': "SymmetricTree",
    'auto_class_weights': 'Balanced',
    'task_type': "CPU",
    "eval_metric": F1Metric(),
 }
model_second = train_final_model(X_second[indiv_selected_features_2], y_second, cat_features=['q'], params=params)
model_second.save_model('second_questions_model.cbm')

In [ ]:
params = {
    "objective": "Logloss",
    'iterations': 225, 
    'colsample_bylevel': 0.53262721224417, 
    'depth': 9, 
    'l2_leaf_reg': 9,
    'boosting_type': 'Plain', 
    'bootstrap_type': 'MVS', 
    'random_strength': 1, 
    'min_data_in_leaf': 310,
    'grow_policy': "SymmetricTree",
    'auto_class_weights': 'Balanced',
    'task_type': "CPU",
    "eval_metric": F1Metric(),
 }

model_third = train_final_model(X_third[indiv_selected_features_2], y_third, cat_features=['q'], params=params)
model_third.save_model('third_questions_model.cbm')